In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x) 

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 8

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def g(x):  
    x_temp = torch.cos(pi*x)
    g_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return g_x

In [10]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda()  
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda()  
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda() 
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda() 
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda() 
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda() 
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda() 
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda() 
    xd2[:, 3] = torch.ones(Nb)
    xe1 = torch.rand(Nb, dimension).cuda()  
    xe1[:, 4] = torch.zeros(Nb)
    xe2 = torch.rand(Nb, dimension).cuda()  
    xe2[:, 4] = torch.ones(Nb)
    xf1 = torch.rand(Nb, dimension).cuda() 
    xf1[:, 5] = torch.zeros(Nb)
    xf2 = torch.rand(Nb, dimension).cuda() 
    xf2[:, 5] = torch.ones(Nb)
    xg1 = torch.rand(Nb, dimension).cuda() 
    xg1[:, 6] = torch.zeros(Nb)
    xg2 = torch.rand(Nb, dimension).cuda() 
    xg2[:, 6] = torch.ones(Nb)
    xh1 = torch.rand(Nb, dimension).cuda() 
    xh1[:, 7] = torch.zeros(Nb)
    xh2 = torch.rand(Nb, dimension).cuda() 
    xh2[:, 7] = torch.ones(Nb)
    
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx4[:, 3] = torch.ones(xb1.size()[0])
    dx5 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx5[:, 4] = torch.ones(xb1.size()[0])
    dx6 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx6[:, 5] = torch.ones(xb1.size()[0])
    dx7 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx7[:, 6] = torch.ones(xb1.size()[0])
    dx8 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx8[:, 7] = torch.ones(xb1.size()[0])
    
    uxa1 = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 * (1.0) + model(xa1) - g(xa1))**2).cuda()
    uxa2 = (((model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2 * (-1.0) + model(xa2) - g(xa2))**2).cuda()
    uxb1 = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 * (1.0) + model(xb1) - g(xb1))**2).cuda()
    uxb2 = (((model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2 * (-1.0) + model(xb2) - g(xb2))**2).cuda()
    uxc1 = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 * (1.0) + model(xc1) - g(xc1))**2).cuda()
    uxc2 = (((model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2 * (-1.0) + model(xc2) - g(xc2))**2).cuda()
    uxd1 = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 * (1.0) + model(xd1) - g(xd1))**2).cuda()
    uxd2 = (((model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2 * (-1.0) + model(xd2) - g(xd2))**2).cuda()
    uxe1 = (((model(xe1+step_size*dx5) - model(xe1-step_size*dx5))/step_size/2 * (1.0) + model(xe1) - g(xe1))**2).cuda()
    uxe2 = (((model(xe2+step_size*dx5) - model(xe2-step_size*dx5))/step_size/2 * (-1.0) + model(xe2) - g(xe2))**2).cuda()
    uxf1 = (((model(xf1+step_size*dx6) - model(xf1-step_size*dx6))/step_size/2 * (1.0) + model(xf1) - g(xf1))**2).cuda()
    uxf2 = (((model(xf2+step_size*dx6) - model(xf2-step_size*dx6))/step_size/2 * (-1.0) + model(xf2) - g(xf2))**2).cuda()
    uxg1 = (((model(xg1+step_size*dx7) - model(xg1-step_size*dx7))/step_size/2 * (1.0) + model(xg1) - g(xg1))**2).cuda()
    uxg2 = (((model(xg2+step_size*dx7) - model(xg2-step_size*dx7))/step_size/2 * (-1.0) + model(xg2) - g(xg2))**2).cuda()
    uxh1 = (((model(xh1+step_size*dx8) - model(xh1-step_size*dx8))/step_size/2 * (1.0) + model(xh1) - g(xh1))**2).cuda()
    uxh2 = (((model(xh2+step_size*dx8) - model(xh2-step_size*dx8))/step_size/2 * (-1.0) + model(xh2) - g(xh2))**2).cuda()
    
    part_2 = torch.sum((uxa1 + uxa2 + uxb1 + uxb2 + uxc1 + uxc2 + uxd1 + uxd2 + uxe1 + uxe2 + uxf1 + uxf2 + uxg1 + uxg2 + uxh1 + uxh2)[:,0])/xa1.size()[0]
    lambda1 = 10.0
    return part_1 + lambda1 * part_2 / 16

In [11]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [12]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [13]:
model = DeepRitzNet(dimension, 16)
model = initparam(model, 0.5)

In [14]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=8, out_features=16, bias=True)
  (linear1): Linear(in_features=16, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=16, bias=True)
  (linear3): Linear(in_features=16, out_features=16, bias=True)
  (linear4): Linear(in_features=16, out_features=16, bias=True)
  (linear5): Linear(in_features=16, out_features=16, bias=True)
  (linear6): Linear(in_features=16, out_features=16, bias=True)
  (linear_out): Linear(in_features=16, out_features=1, bias=True)
)

In [15]:
import torch.optim as optim
import torch.nn as nn
import time

In [16]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [17]:
traintime = 50000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [18]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_8D_Robin.npy", error_save)
np.save("DRM_relative_error_8D_Robin.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(5050.3904, device='cuda:0')
current relative error is:  tensor(8.9442, device='cuda:0')
current epoch is:  50
current loss is:  tensor(177.4145, device='cuda:0')
current relative error is:  tensor(1.5560, device='cuda:0')
current epoch is:  100
current loss is:  tensor(72.4863, device='cuda:0')
current relative error is:  tensor(1.0672, device='cuda:0')
current epoch is:  150
current loss is:  tensor(33.9806, device='cuda:0')
current relative error is:  tensor(0.8472, device='cuda:0')
current epoch is:  200
current loss is:  tensor(19.0458, device='cuda:0')
current relative error is:  tensor(0.7248, device='cuda:0')
current epoch is:  250
current loss is:  tensor(8.6468, device='cuda:0')
current relative error is:  tensor(0.6478, device='cuda:0')
current epoch is:  300
current loss is:  tensor(3.5683, device='cuda:0')
current relative error is:  tensor(0.5841, device='cuda:0')
current epoch is:  350
current loss is:  tensor(-2.5690, device=

current epoch is:  3050
current loss is:  tensor(-35.6088, device='cuda:0')
current relative error is:  tensor(0.1400, device='cuda:0')
current epoch is:  3100
current loss is:  tensor(-35.3329, device='cuda:0')
current relative error is:  tensor(0.1650, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(-33.0000, device='cuda:0')
current relative error is:  tensor(0.1569, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(-37.1999, device='cuda:0')
current relative error is:  tensor(0.1391, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(-32.5518, device='cuda:0')
current relative error is:  tensor(0.1574, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(-39.5344, device='cuda:0')
current relative error is:  tensor(0.1328, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(-35.0911, device='cuda:0')
current relative error is:  tensor(0.1420, device='cuda:0')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(-38.7768, device='cuda:0')
current relative error is:  tensor(0.0705, device='cuda:0')
current epoch is:  6150
current loss is:  tensor(-36.9508, device='cuda:0')
current relative error is:  tensor(0.0946, device='cuda:0')
current epoch is:  6200
current loss is:  tensor(-38.5851, device='cuda:0')
current relative error is:  tensor(0.0563, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(-39.1455, device='cuda:0')
current relative error is:  tensor(0.0648, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(-39.1554, device='cuda:0')
current relative error is:  tensor(0.0563, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(-37.3579, device='cuda:0')
current relative error is:  tensor(0.0555, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(-41.3915, device='cuda:0')
current relative error is:  tensor(0.0856, device='cuda:0')
current epoch is:  6450
current loss is:  tensor

current epoch is:  9150
current loss is:  tensor(-40.7259, device='cuda:0')
current relative error is:  tensor(0.0470, device='cuda:0')
current epoch is:  9200
current loss is:  tensor(-41.0478, device='cuda:0')
current relative error is:  tensor(0.0652, device='cuda:0')
current epoch is:  9250
current loss is:  tensor(-41.5902, device='cuda:0')
current relative error is:  tensor(0.0626, device='cuda:0')
current epoch is:  9300
current loss is:  tensor(-37.8283, device='cuda:0')
current relative error is:  tensor(0.0442, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(-40.2272, device='cuda:0')
current relative error is:  tensor(0.0398, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(-36.1281, device='cuda:0')
current relative error is:  tensor(0.0949, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(-39.6847, device='cuda:0')
current relative error is:  tensor(0.0623, device='cuda:0')
current epoch is:  9500
current loss is:  tensor

current epoch is:  12150
current loss is:  tensor(-37.0767, device='cuda:0')
current relative error is:  tensor(0.0857, device='cuda:0')
current epoch is:  12200
current loss is:  tensor(-39.2595, device='cuda:0')
current relative error is:  tensor(0.0373, device='cuda:0')
current epoch is:  12250
current loss is:  tensor(-41.0412, device='cuda:0')
current relative error is:  tensor(0.0360, device='cuda:0')
current epoch is:  12300
current loss is:  tensor(-40.0711, device='cuda:0')
current relative error is:  tensor(0.0348, device='cuda:0')
current epoch is:  12350
current loss is:  tensor(-37.3051, device='cuda:0')
current relative error is:  tensor(0.0455, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(-37.9538, device='cuda:0')
current relative error is:  tensor(0.0436, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(-36.6644, device='cuda:0')
current relative error is:  tensor(0.0343, device='cuda:0')
current epoch is:  12500
current loss is:

...

current epoch is:  36150
current loss is:  tensor(-40.3590, device='cuda:0')
current relative error is:  tensor(0.0694, device='cuda:0')
current epoch is:  36200
current loss is:  tensor(-40.2490, device='cuda:0')
current relative error is:  tensor(0.0800, device='cuda:0')
current epoch is:  36250
current loss is:  tensor(-42.1907, device='cuda:0')
current relative error is:  tensor(0.0219, device='cuda:0')
current epoch is:  36300
current loss is:  tensor(-40.2010, device='cuda:0')
current relative error is:  tensor(0.0632, device='cuda:0')
current epoch is:  36350
current loss is:  tensor(-37.1886, device='cuda:0')
current relative error is:  tensor(0.0203, device='cuda:0')
current epoch is:  36400
current loss is:  tensor(-41.4911, device='cuda:0')
current relative error is:  tensor(0.0444, device='cuda:0')
current epoch is:  36450
current loss is:  tensor(-38.3392, device='cuda:0')
current relative error is:  tensor(0.0174, device='cuda:0')
current epoch is:  36500
current loss is:

current epoch is:  39150
current loss is:  tensor(-39.4274, device='cuda:0')
current relative error is:  tensor(0.0239, device='cuda:0')
current epoch is:  39200
current loss is:  tensor(-38.6968, device='cuda:0')
current relative error is:  tensor(0.0346, device='cuda:0')
current epoch is:  39250
current loss is:  tensor(-35.7352, device='cuda:0')
current relative error is:  tensor(0.0371, device='cuda:0')
current epoch is:  39300
current loss is:  tensor(-39.5401, device='cuda:0')
current relative error is:  tensor(0.0366, device='cuda:0')
current epoch is:  39350
current loss is:  tensor(-37.5044, device='cuda:0')
current relative error is:  tensor(0.0303, device='cuda:0')
current epoch is:  39400
current loss is:  tensor(-38.2804, device='cuda:0')
current relative error is:  tensor(0.0234, device='cuda:0')
current epoch is:  39450
current loss is:  tensor(-37.8596, device='cuda:0')
current relative error is:  tensor(0.0166, device='cuda:0')
current epoch is:  39500
current loss is:

current epoch is:  42150
current loss is:  tensor(-43.1356, device='cuda:0')
current relative error is:  tensor(0.0250, device='cuda:0')
current epoch is:  42200
current loss is:  tensor(-38.7842, device='cuda:0')
current relative error is:  tensor(0.0491, device='cuda:0')
current epoch is:  42250
current loss is:  tensor(-39.8326, device='cuda:0')
current relative error is:  tensor(0.0230, device='cuda:0')
current epoch is:  42300
current loss is:  tensor(-39.4346, device='cuda:0')
current relative error is:  tensor(0.0182, device='cuda:0')
current epoch is:  42350
current loss is:  tensor(-42.4456, device='cuda:0')
current relative error is:  tensor(0.0396, device='cuda:0')
current epoch is:  42400
current loss is:  tensor(-38.4562, device='cuda:0')
current relative error is:  tensor(0.0506, device='cuda:0')
current epoch is:  42450
current loss is:  tensor(-40.1751, device='cuda:0')
current relative error is:  tensor(0.0259, device='cuda:0')
current epoch is:  42500
current loss is:

current epoch is:  45150
current loss is:  tensor(-39.9183, device='cuda:0')
current relative error is:  tensor(0.0186, device='cuda:0')
current epoch is:  45200
current loss is:  tensor(-42.0215, device='cuda:0')
current relative error is:  tensor(0.0172, device='cuda:0')
current epoch is:  45250
current loss is:  tensor(-36.5562, device='cuda:0')
current relative error is:  tensor(0.0279, device='cuda:0')
current epoch is:  45300
current loss is:  tensor(-37.9708, device='cuda:0')
current relative error is:  tensor(0.0268, device='cuda:0')
current epoch is:  45350
current loss is:  tensor(-39.3899, device='cuda:0')
current relative error is:  tensor(0.0156, device='cuda:0')
current epoch is:  45400
current loss is:  tensor(-41.5067, device='cuda:0')
current relative error is:  tensor(0.0511, device='cuda:0')
current epoch is:  45450
current loss is:  tensor(-35.6007, device='cuda:0')
current relative error is:  tensor(0.0352, device='cuda:0')
current epoch is:  45500
current loss is:

current epoch is:  48150
current loss is:  tensor(-38.3359, device='cuda:0')
current relative error is:  tensor(0.0576, device='cuda:0')
current epoch is:  48200
current loss is:  tensor(-39.1710, device='cuda:0')
current relative error is:  tensor(0.0196, device='cuda:0')
current epoch is:  48250
current loss is:  tensor(-38.9235, device='cuda:0')
current relative error is:  tensor(0.0221, device='cuda:0')
current epoch is:  48300
current loss is:  tensor(-38.5900, device='cuda:0')
current relative error is:  tensor(0.0208, device='cuda:0')
current epoch is:  48350
current loss is:  tensor(-38.5866, device='cuda:0')
current relative error is:  tensor(0.0299, device='cuda:0')
current epoch is:  48400
current loss is:  tensor(-39.6950, device='cuda:0')
current relative error is:  tensor(0.0313, device='cuda:0')
current epoch is:  48450
current loss is:  tensor(-39.3910, device='cuda:0')
current relative error is:  tensor(0.0182, device='cuda:0')
current epoch is:  48500
current loss is:

In [19]:
x_test = torch.rand(10, 8)
torch.abs(model(x) - u_ex(x))

tensor([[0.0375],
        [0.0425],
        [0.0010],
        ...,
        [0.0134],
        [0.0322],
        [0.0663]], device='cuda:0', grad_fn=<AbsBackward>)